In [62]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
print("tensorflow version", tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import sys


tensorflow version 2.9.1


In [63]:
cwd = os.getcwd()
cwd

'/Users/wuming/Desktop/22 SP/Info 371/Labs'

In [64]:
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/Users/wuming/Desktop/22 SP/Info 371/Labs': ['lab04.html', 'lab03.ipynb', 'lab00-test-r-rmd.rmd', '.DS_Store', 'lab07.ipynb', 'lab05.ipynb', 'lab07-nnets.pdf', 'lab05.html', 'lab06-trees.pdf', 'lab02.Rmd', 'lab02.html', 'Lab01.Rmd', 'lab01-ba-cs.pdf', 'lab08-squares-circles.py', 'lab03.html', 'lab04.ipynb', 'Lab Demo', 'lab01-ba-cs.Rmd', 'lab06.ipynb', 'squares-circles', 'lab08-image-recognition.pdf', 'squares-circles.zip', 'lab03-knn.pdf', 'lab08.ipynb', 'lab00-test-python-notebooks.html', 'lab00-test-r-rmd.html', 'lab02-did.pdf', '.ipynb_checkpoints', 'lab06.html', 'lab05-bayes-predict.pdf', 'lab04-pca.pdf', 'lab01-ba-cs.html', 'lab00-test-python-notebooks.ipynb', 'lab07.html']


In [73]:
## Define image properties:
imgDir = "squares-circles"
targetWidth, targetHeight = 35, 35
imageSize = (targetWidth, targetHeight)
channels = 1  # color channels black/white
## define other constants, including command line argument defaults
epochs = 10
plot = False # show plots?

In [74]:
## command line arguments
# check if this was run as a separate file (not inside notebook)
import __main__ as main
if hasattr(main, "__file__"):
    # run as file
    print("parsing command line arguments")
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--dir", "-d",
                        help = "directory to read images from",
                        default = imgDir)
    parser.add_argument("--epochs", "-e",
                        help = "how many epochs",
                        default= epochs)
    parser.add_argument("--plot", "-p",
                        action = "store_true",
                        help = "plot a few wrong/correct results")
    args = parser.parse_args()
    imgDir = args.dir
    epochs = int(args.epochs)
    plot = args.plot
else:
    # run as notebook
    print("run interactively from", os.getcwd())
    imageDir = os.path.join(os.path.expanduser("~"),
                            "data", "images", "text", "language-text-images")
print("Load images from", imgDir)
print("epochs:", epochs)

run interactively from /Users/wuming/Desktop/22 SP/Info 371/Labs
Load images from squares-circles
epochs: 10


In [75]:
## Prepare dataset for training model:
# filenames = filter(lambda x: '.i' not in x, os.listdir(os.path.join(imgDir, "train")))
filenames = os.listdir(os.path.join(imgDir, "train"))
print(len(filenames), "images found")
trainingResults = pd.DataFrame({
    'filename':filenames,
    'category':pd.Series(filenames).str[:2]
})
print("data files:")
print(trainingResults.sample(5))
nCategories = trainingResults.category.nunique()
print("categories:\n", trainingResults.category.value_counts())
## Create model
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,\
    MaxPooling2D, AveragePooling2D,\
    Dropout,Flatten,Dense,Activation,\
    BatchNormalization

# sequential (not recursive) model (one input, one output)
model=Sequential()

model.add(Conv2D(32,
                 kernel_size=3,
                 strides=2,
                 activation='relu',
                 kernel_initializer = initializers.HeNormal(),
                 input_shape=(targetWidth, targetHeight, channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=3))
model.add(Dropout(0.25))

model.add(Conv2D(64,
                 kernel_size=3,
                 kernel_initializer = initializers.HeNormal(),
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,
                kernel_initializer = initializers.HeNormal(),
                activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.add(Dense(nCategories,
                kernel_initializer = initializers.HeNormal(),
                activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

4000 images found
data files:
        filename category
1652  ci2815.jpg       ci
2573  ci0548.jpg       ci
2769  sq3911.jpg       sq
2155  sq4464.jpg       sq
1343  ci2435.jpg       ci
categories:
 ci    2028
sq    1972
Name: category, dtype: int64
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 17, 17, 32)        320       
                                                                 
 batch_normalization_24 (Bat  (None, 17, 17, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 5, 5, 32)         0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 5, 5, 32)          0         
  

In [77]:
## Training and validation data generator:
trainingGenerator = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
).\
    flow_from_dataframe(trainingResults,
                        os.path.join(imgDir, "train"),
                        x_col='filename', y_col='category',
                        target_size=imageSize,
                        class_mode='categorical',
                        color_mode="grayscale",
                        shuffle=True)
label_map = trainingGenerator.class_indices
## Model Training:
history = model.fit(
    trainingGenerator,
    epochs=epochs
)

## Validation data preparation:
validationDir = os.path.join(imgDir, "validation")
fNames = os.listdir(validationDir)
print(len(fNames), "validation images")
validationResults = pd.DataFrame({
    'filename': fNames,
    'category': pd.Series(fNames).str[:2]
})
print(validationResults.shape[0], "validation files read from", validationDir)
validationGenerator = ImageDataGenerator(rescale=1./255).\
    flow_from_dataframe(validationResults,
                        os.path.join(imgDir, "validation"),
                        x_col='filename',
                        class_mode = None,
                        target_size = imageSize,
                        shuffle = False,
                        # do _not_ randomize the order!
                        # this would clash with the file name order!
                        color_mode="grayscale"
    )

## Make categorical prediction:
print(" --- Predicting on validation data ---")
phat = model.predict(validationGenerator)
print("Predicted probability array shape:", phat.shape)
print("Example:\n", phat[:5])

## Convert labels to categories:
validationResults['predicted'] = pd.Series(np.argmax(phat, axis=-1), index=validationResults.index)
print(validationResults.head())
labelMap = {v: k for k, v in label_map.items()}
validationResults["predicted"] = validationResults.predicted.replace(labelMap)
print("confusion matrix (validation)")
print(pd.crosstab(validationResults.category, validationResults.predicted))
print("Validation accuracy", np.mean(validationResults.category == validationResults.predicted))

## Print and plot misclassified results
wrongResults = validationResults[validationResults.predicted != validationResults.category]
rows = np.random.choice(wrongResults.index, min(4, wrongResults.shape[0]), replace=False)
print("Example wrong results (validation data)")
print(wrongResults.sample(min(10, wrongResults.shape[0])))
if plot:
    plt.figure(figsize=(12, 12))
    index = 1
    for row in rows:
        filename = wrongResults.loc[row, 'filename']
        predicted = wrongResults.loc[row, 'predicted']
        img = load_img(os.path.join(imgDir, "validation", filename), target_size=imageSize)
        plt.subplot(4, 2, index)
        plt.imshow(img)
        plt.xlabel(filename + " ({})".format(predicted))
        index += 1
    # now show correct results
    index = 5
    correctResults = validationResults[validationResults.predicted == validationResults.category]
    rows = np.random.choice(correctResults.index,
                            min(4, correctResults.shape[0]), replace=False)
    for row in rows:
        filename = correctResults.loc[row, 'filename']
        predicted = correctResults.loc[row, 'predicted']
        img = load_img(os.path.join(imgDir, "validation", filename), target_size=imageSize)
        plt.subplot(4, 2, index)
        plt.imshow(img)
        plt.xlabel(filename + " ({})".format(predicted))
        index += 1
    plt.tight_layout()
    plt.show()

## Training data predictions.
## Do these here to keep the in place for students
## 
print(" --- Predicting on training data: ---")
# do another generator: the same as training, just w/o shuffle
predictTrainGenerator = ImageDataGenerator(rescale=1./255).\
    flow_from_dataframe(trainingResults,
                        os.path.join(imgDir, "train"),
                        x_col='filename', y_col='category',
                        target_size=imageSize,
                        class_mode='categorical',
                        color_mode="grayscale",
                        shuffle=False  # do not shuffle!
    )
phat = model.predict(predictTrainGenerator)
trainingResults['predicted'] = pd.Series(np.argmax(phat, axis=-1), index=trainingResults.index)
trainingResults["predicted"] = trainingResults.predicted.replace(labelMap)
print("confusion matrix (training)")
print(pd.crosstab(trainingResults.category, trainingResults.predicted))
print("Train accuracy", np.mean(trainingResults.category == trainingResults.predicted))

Found 4000 validated image filenames belonging to 2 classes.
Epoch 1/10
125/125 [==============================] - 7s 45ms/step - loss: 0.5963 - accuracy: 0.7387
Epoch 2/10
125/125 [==============================] - 4s 33ms/step - loss: 0.4493 - accuracy: 0.8487
Epoch 3/10
125/125 [==============================] - 4s 31ms/step - loss: 0.3884 - accuracy: 0.8717
Epoch 4/10
125/125 [==============================] - 4s 30ms/step - loss: 0.3324 - accuracy: 0.8960
Epoch 5/10
125/125 [==============================] - 4s 31ms/step - loss: 0.2981 - accuracy: 0.9093
Epoch 6/10
125/125 [==============================] - 4s 30ms/step - loss: 0.2775 - accuracy: 0.9150
Epoch 7/10
125/125 [==============================] - 4s 32ms/step - loss: 0.2495 - accuracy: 0.9277
Epoch 8/10
125/125 [==============================] - 4s 30ms/step - loss: 0.2494 - accuracy: 0.9233
Epoch 9/10
125/125 [==============================] - 4s 31ms/step - loss: 0.2302 - accuracy: 0.9302
Epoch 10/10
125/125 [=========

In [78]:
## Prepare dataset for training model:
filenames = os.listdir(os.path.join(imgDir, "train"))
print(len(filenames), "images found")
trainingResults = pd.DataFrame({
    'filename':filenames,
    'category':pd.Series(filenames).str[:2]
})
print("data files:")
print(trainingResults.sample(5))
nCategories = trainingResults.category.nunique()
print("categories:\n", trainingResults.category.value_counts())
## Create model
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,\
    MaxPooling2D, AveragePooling2D,\
    Dropout,Flatten,Dense,Activation,\
    BatchNormalization

# sequential (not recursive) model (one input, one output)
model=Sequential()

model.add(Conv2D(16,
                 kernel_size=2,
                 strides=1,
                 activation='relu',
                 kernel_initializer = initializers.HeNormal(),
                 input_shape=(targetWidth, targetHeight, channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=3))
model.add(Dropout(0.25))

model.add(Conv2D(16,
                 kernel_size=3,
                 kernel_initializer = initializers.HeNormal(),
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(8,
                kernel_initializer = initializers.HeNormal(),
                activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.add(Dense(nCategories,
                kernel_initializer = initializers.HeNormal(),
                activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

4000 images found
data files:
        filename category
2265  sq4277.jpg       sq
1632  ci3504.jpg       ci
196   sq1601.jpg       sq
1941  ci3506.jpg       ci
1412  ci3299.jpg       ci
categories:
 ci    2028
sq    1972
Name: category, dtype: int64
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 34, 34, 16)        80        
                                                                 
 batch_normalization_27 (Bat  (None, 34, 34, 16)       64        
 chNormalization)                                                
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 11, 11, 16)       0         
 g2D)                                                            
                                                                 
 dropout_27 (Dropout)        (None, 11, 11, 16)        0         
  

In [69]:
## Training and validation data generator:
trainingGenerator = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
).\
    flow_from_dataframe(trainingResults,
                        os.path.join(imgDir, "train"),
                        x_col='filename', y_col='category',
                        target_size=imageSize,
                        class_mode='categorical',
                        color_mode="grayscale",
                        shuffle=True)
label_map = trainingGenerator.class_indices


Found 4000 validated image filenames belonging to 2 classes.


In [70]:
## Model Training:
history = model.fit(
    trainingGenerator,
    epochs=epochs
)

Epoch 1/10
125/125 [==============================] - 8s 46ms/step - loss: 0.5506 - accuracy: 0.7550
Epoch 2/10
125/125 [==============================] - 4s 32ms/step - loss: 0.4473 - accuracy: 0.8407
Epoch 3/10
125/125 [==============================] - 4s 33ms/step - loss: 0.3888 - accuracy: 0.8730
Epoch 4/10
125/125 [==============================] - 4s 31ms/step - loss: 0.3395 - accuracy: 0.8978
Epoch 5/10
125/125 [==============================] - 4s 32ms/step - loss: 0.3052 - accuracy: 0.9112
Epoch 6/10
125/125 [==============================] - 4s 31ms/step - loss: 0.2750 - accuracy: 0.9200
Epoch 7/10
125/125 [==============================] - 4s 29ms/step - loss: 0.2591 - accuracy: 0.9225
Epoch 8/10
125/125 [==============================] - 4s 29ms/step - loss: 0.2458 - accuracy: 0.9270
Epoch 9/10
125/125 [==============================] - 4s 30ms/step - loss: 0.2142 - accuracy: 0.9392
Epoch 10/10
125/125 [==============================] - 4s 30ms/step - loss: 0.2247 - accura

In [71]:
## Validation data preparation:
validationDir = os.path.join(imgDir, "validation")
fNames = os.listdir(validationDir)
print(len(fNames), "validation images")
validationResults = pd.DataFrame({
    'filename': fNames,
    'category': pd.Series(fNames).str[:2]
})
print(validationResults.shape[0], "validation files read from", validationDir)
validationGenerator = ImageDataGenerator(rescale=1./255).\
    flow_from_dataframe(validationResults,
                        os.path.join(imgDir, "validation"),
                        x_col='filename',
                        class_mode = None,
                        target_size = imageSize,
                        shuffle = False,
                        # do _not_ randomize the order!
                        # this would clash with the file name order!
                        color_mode="grayscale"
    )

1000 validation images
1000 validation files read from squares-circles/validation
Found 1000 validated image filenames.


In [72]:

## Make categorical prediction:
print(" --- Predicting on validation data ---")
phat = model.predict(validationGenerator)
print("Predicted probability array shape:", phat.shape)
print("Example:\n", phat[:5])

## Convert labels to categories:
validationResults['predicted'] = pd.Series(np.argmax(phat, axis=-1), index=validationResults.index)
print(validationResults.head())
labelMap = {v: k for k, v in label_map.items()}
validationResults["predicted"] = validationResults.predicted.replace(labelMap)
print("confusion matrix (validation)")
print(pd.crosstab(validationResults.category, validationResults.predicted))
print("Validation accuracy", np.mean(validationResults.category == validationResults.predicted))

## Print and plot misclassified results
wrongResults = validationResults[validationResults.predicted != validationResults.category]
rows = np.random.choice(wrongResults.index, min(4, wrongResults.shape[0]), replace=False)
print("Example wrong results (validation data)")
print(wrongResults.sample(min(10, wrongResults.shape[0])))
if plot:
    plt.figure(figsize=(12, 12))
    index = 1
    for row in rows:
        filename = wrongResults.loc[row, 'filename']
        predicted = wrongResults.loc[row, 'predicted']
        img = load_img(os.path.join(imgDir, "validation", filename), target_size=imageSize)
        plt.subplot(4, 2, index)
        plt.imshow(img)
        plt.xlabel(filename + " ({})".format(predicted))
        index += 1
    # now show correct results
    index = 5
    correctResults = validationResults[validationResults.predicted == validationResults.category]
    rows = np.random.choice(correctResults.index,
                            min(4, correctResults.shape[0]), replace=False)
    for row in rows:
        filename = correctResults.loc[row, 'filename']
        predicted = correctResults.loc[row, 'predicted']
        img = load_img(os.path.join(imgDir, "validation", filename), target_size=imageSize)
        plt.subplot(4, 2, index)
        plt.imshow(img)
        plt.xlabel(filename + " ({})".format(predicted))
        index += 1
    plt.tight_layout()
    plt.show()

## Training data predictions.
## Do these here to keep the in place for students
## 
print(" --- Predicting on training data: ---")
# do another generator: the same as training, just w/o shuffle
predictTrainGenerator = ImageDataGenerator(rescale=1./255).\
    flow_from_dataframe(trainingResults,
                        os.path.join(imgDir, "train"),
                        x_col='filename', y_col='category',
                        target_size=imageSize,
                        class_mode='categorical',
                        color_mode="grayscale",
                        shuffle=False  # do not shuffle!
    )
phat = model.predict(predictTrainGenerator)
trainingResults['predicted'] = pd.Series(np.argmax(phat, axis=-1), index=trainingResults.index)
trainingResults["predicted"] = trainingResults.predicted.replace(labelMap)
print("confusion matrix (training)")
print(pd.crosstab(trainingResults.category, trainingResults.predicted))
print("Train accuracy", np.mean(trainingResults.category == trainingResults.predicted))

 --- Predicting on validation data ---
32/32 [==============================] - 1s 26ms/step
Predicted probability array shape: (1000, 2)
Example:
 [[0.19727843 0.80272156]
 [0.9231539  0.07684609]
 [0.9230086  0.07699139]
 [0.9231477  0.07685231]
 [0.92315376 0.07684622]]
     filename category  predicted
0  ci0134.jpg       ci          1
1  ci2723.jpg       ci          0
2  ci4434.jpg       ci          0
3  ci4346.jpg       ci          0
4  ci1216.jpg       ci          0
confusion matrix (validation)
predicted   ci   sq
category           
ci         455   50
sq           1  494
Validation accuracy 0.949
Example wrong results (validation data)
       filename category predicted
147  ci4181.jpg       ci        sq
492  ci1073.jpg       ci        sq
152  ci3771.jpg       ci        sq
380  ci0810.jpg       ci        sq
86   ci3774.jpg       ci        sq
345  ci4290.jpg       ci        sq
32   ci3213.jpg       ci        sq
0    ci0134.jpg       ci        sq
582  ci3859.jpg       ci       